In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
import os
import cv2
import re
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.backend import int_shape
from tensorflow.keras import Model, Input, Sequential
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, UpSampling2D, Add, BatchNormalization, \
    Input, Activation, Lambda, Concatenate, Embedding, Flatten, Multiply, Reshape, LeakyReLU, Conv2DTranspose, Dropout
from tensorflow.python.keras.utils.vis_utils import plot_model

In [2]:
# Free up RAM in case the model definition cells were run multiple times
keras.backend.clear_session()

In [13]:
z_dim = 100
num_class = 9
img_shape = (28,28,1)
batch_size = 32
epochs = 10
sample_interval = 5

In [4]:
def bgr_to_rgb(img):
    b,g,r = cv2.split(img)
    return cv2.merge([r,g,b])

def get_label(file: str, labels: dict):
    match = re.search('[a-z]+', file).group(0)
    return labels[match]

def load_all_imgs(path, img_size, dict_label, mode):
    imgs = []
    labels = []
    files = os.listdir(path)
    files.sort()
    print(f'\n{path} found {len(files)} img to load')
    for index, file in enumerate(files):
        img_path = os.path.join(path, file)
        img = cv2.imread(img_path, mode)
        
        if mode == 0:
            img = np.expand_dims(img, 2)
        
        if mode == 1:
            img = bgr_to_rgb(img)
        img = cv2.resize(img, img_size)
        img = img / 255.
        imgs.append(img)
        labels.append(get_label(file, dict_label))
#         if index % 100 == 0:
#             print(f'\n[{index}]:', end='')
#         else:
#             print("|", end='')
    return np.array(imgs), np.array(labels)

In [5]:
classes = {'healthy': 0, 'mild': 1, 'moderate': 2, 'severe': 3}
img, labels = load_all_imgs('../../dataset/64x64/dataset_64x64/', img_shape[:2], classes, 1)


../../dataset/64x64/dataset_64x64/ found 6194 img to load


In [37]:
from tensorflow.keras.datasets import fashion_mnist
(x_train, y_train), (x_test, y_test)  = fashion_mnist.load_data()
img = np.concatenate([x_train, x_test])
labels = np.concatenate([y_train, y_test])

img = img.astype("float32") / 255.0
img = np.reshape(img, (-1, 28, 28, 1))

In [23]:
def build_generator(z_dim):
    model = Sequential(name='generator')
    # foundation for 4x4 image
    n_nodes = 512 * 7 * 7
    model.add(Dense(n_nodes, input_dim=z_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((7, 7, 512)))
    model.add(Conv2D(512, kernel_size=3, padding='same', name="Conv2D_1"))
    model.add(BatchNormalization(name="BN_1"))
    model.add(Activation('relu', name='Act_1'))
    model.add(Conv2DTranspose(256, kernel_size=2, strides=1, padding='same'))
    model.add(Conv2D(256, kernel_size=3, padding='same', name="Conv2D_2"))
    model.add(BatchNormalization(name="BN_2"))
    model.add(Activation('relu', name='Act_2'))
    model.add(Conv2D(256, kernel_size=3, padding='same', name="Conv2D_3"))
    model.add(BatchNormalization(name="BN_3"))
    model.add(Activation('relu', name='Act_3'))
    model.add(Conv2DTranspose(128, kernel_size=2, strides=(2, 2), padding='same'))
    model.add(Conv2D(128, kernel_size=3, padding='same', name="Conv2D_4"))
    model.add(BatchNormalization(name="BN_4"))
    model.add(Activation('relu', name='Act_4'))
    model.add(Conv2D(128, kernel_size=3, padding='same', name="Conv2D_5"))
    model.add(BatchNormalization(name="BN_5"))
    model.add(Activation('relu', name='Act_5'))
    model.add(Conv2DTranspose(64, kernel_size=2, strides=1, padding='same'))
    model.add(Conv2D(64, kernel_size=3, padding='same', name="Conv2D_6"))
    model.add(BatchNormalization(name="BN_6"))
    model.add(Activation('relu', name='Act_6'))
    model.add(Conv2D(64, kernel_size=3, padding='same', name="Conv2D_7"))
    model.add(BatchNormalization(name="BN_7"))
    model.add(Activation('relu', name='Act_7'))
    model.add(Conv2DTranspose(32, kernel_size=2, strides=(2, 2), padding='same'))
    model.add(Conv2D(32, kernel_size=3, padding='same', name="Conv2D_8"))
    model.add(BatchNormalization(name="BN_8"))
    model.add(Activation('relu', name='Act_8'))
    model.add(Conv2D(32, kernel_size=3, padding='same', name="Conv2D_9"))
    model.add(BatchNormalization(name="BN_9"))
    model.add(Activation('relu', name='Act_9'))
    model.add(Conv2D(1, kernel_size=1, padding='same', activation='tanh', name='output'))

    return model


def build_cgan_generator(z_dim, num_classes):
    # latent input
    z = Input(shape=(z_dim,))
    # label input
    label = Input(shape=(1,), dtype='int32')
    # convert label to embedding
    label_embedding = Embedding(num_classes, z_dim)(label)

    label_embedding = Flatten()(label_embedding)
    # dot product two inputs
    joined_representation = Multiply()([z, label_embedding])

    generator = build_generator(z_dim)

    conditioned_img = generator(joined_representation)

    model = Model([z, label], conditioned_img)
    # save model blueprint to image
    plot_model(model, './cgan_generator.jpg', show_shapes=True, show_dtype=True)

    return model

In [24]:
generator = build_cgan_generator(z_dim, num_class)
generator.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 1, 100)       900         input_13[0][0]                   
__________________________________________________________________________________________________
input_12 (InputLayer)           [(None, 100)]        0                                            
__________________________________________________________________________________________________
flatten_5 (Flatten)             (None, 100)          0           embedding_4[0][0]                
____________________________________________________________________________________________

In [25]:
generator.get_layer('generator').summary()

Model: "generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 25088)             2533888   
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 25088)             0         
_________________________________________________________________
reshape_4 (Reshape)          (None, 7, 7, 512)         0         
_________________________________________________________________
Conv2D_1 (Conv2D)            (None, 7, 7, 512)         2359808   
_________________________________________________________________
BN_1 (BatchNormalization)    (None, 7, 7, 512)         2048      
_________________________________________________________________
Act_1 (Activation)           (None, 7, 7, 512)         0         
_________________________________________________________________
conv2d_transpose_12 (Conv2DT (None, 7, 7, 256)         52

In [26]:
def build_discriminator(img_shape):
    model = Sequential(name="discriminator")
    model.add(Input((img_shape[0], img_shape[1], img_shape[2] * 2)))
    model.add(Conv2D(32, kernel_size=3, padding='same', name='D_Conv2D_1'))
    model.add(BatchNormalization(name="D_BN_1"))
    model.add(Activation('relu', name='D_Act_1'))
    model.add(Conv2D(32, kernel_size=3, padding='same', name="D_Conv2D_2"))
    model.add(BatchNormalization(name="D_BN_2"))
    model.add(Activation('relu', name='D_Act_2'))
    model.add(MaxPooling2D(padding='same', name="D_MaxPooling_1"))
    model.add(Conv2D(64, kernel_size=3, padding='same', name='D_Conv2D_3'))
    model.add(BatchNormalization(name="D_BN_3"))
    model.add(Activation('relu', name='D_Act_3'))
    model.add(Conv2D(64, kernel_size=3, padding='same', name="D_Conv2D_4"))
    model.add(BatchNormalization(name="D_BN_4"))
    model.add(Activation('relu', name='D_Act_4'))
    model.add(MaxPooling2D(padding='same', name="D_MaxPooling_2"))
    model.add(Conv2D(128, kernel_size=3, padding='same', name='D_Conv2D_5'))
    model.add(BatchNormalization(name="D_BN_5"))
    model.add(Activation('relu', name='D_Act_5'))
    model.add(Conv2D(128, kernel_size=3, padding='same', name="D_Conv2D_6"))
    model.add(BatchNormalization(name="D_BN_6"))
    model.add(Activation('relu', name='D_Act_6'))
    model.add(MaxPooling2D(padding='same', name="D_MaxPooling_3"))
    model.add(Conv2D(256, kernel_size=3, padding='same', name='D_Conv2D_7'))
    model.add(BatchNormalization(name="D_BN_7"))
    model.add(Activation('relu', name='D_Act_7'))
    model.add(Conv2D(256, kernel_size=3, padding='same', name="D_Conv2D_8"))
    model.add(BatchNormalization(name="D_BN_8"))
    model.add(Activation('relu', name='D_Act_8'))
#     model.add(MaxPooling2D(padding='same', name="MaxPooling_4"))
#     model.add(Conv2D(256, kernel_size=3, padding='same', name='Conv2D_9'))
#     model.add(BatchNormalization(name="BN_9"))
#     model.add(Activation('relu', name='Act_9'))
    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(1, activation='sigmoid'))

    return model


def build_cgan_discriminator(img_shape, num_classes):
    # image input
    img = Input(shape=img_shape)
    # label input
    label = Input(shape=(1,), dtype='int32')

    label_embedding = Embedding(num_classes, np.prod(img_shape), input_length=1)(label)

    label_embedding = Flatten()(label_embedding)

    label_embedding = Reshape(img_shape)(label_embedding)
    # concatenate the image and label
    concatenated = Concatenate(axis=-1)([img, label_embedding])

    discriminator = build_discriminator(img_shape)

    classification = discriminator(concatenated)

    model = Model([img, label], classification)

    plot_model(model, './cgan_discriminator.jpg', show_shapes=True, show_dtype=True)

    return model

In [27]:
threshould = 0.89
metrics = keras.metrics.Accuracy()

def binary_loss(y_true, y_pred):
    j = []
    for i in y_pred:
        if i >= threshould:
            j.append([1.])
        else:
            j.append([0.])
    print(y_pred)
    y_pred = tf.convert_to_tensor(j)
    print(y_true, y_pred)
    loss = keras.losses.binary_crossentropy(y_true, y_pred)
    return loss.numpy()

def accuracy(y_true, y_pred):
    print(y_true, y_pred)
    metrics.reset_states()
    metrics.update_state(y_true, y_pred)
    return metrics.result()

In [28]:
discriminator = build_cgan_discriminator(img_shape, num_class)
discriminator.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [29]:
discriminator.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_15 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, 1, 784)       7056        input_15[0][0]                   
__________________________________________________________________________________________________
flatten_6 (Flatten)             (None, 784)          0           embedding_5[0][0]                
__________________________________________________________________________________________________
input_14 (InputLayer)           [(None, 28, 28, 1)]  0                                            
____________________________________________________________________________________________

In [30]:
discriminator.get_layer('discriminator').summary()

Model: "discriminator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
D_Conv2D_1 (Conv2D)          (None, 28, 28, 32)        608       
_________________________________________________________________
D_BN_1 (BatchNormalization)  (None, 28, 28, 32)        128       
_________________________________________________________________
D_Act_1 (Activation)         (None, 28, 28, 32)        0         
_________________________________________________________________
D_Conv2D_2 (Conv2D)          (None, 28, 28, 32)        9248      
_________________________________________________________________
D_BN_2 (BatchNormalization)  (None, 28, 28, 32)        128       
_________________________________________________________________
D_Act_2 (Activation)         (None, 28, 28, 32)        0         
_________________________________________________________________
D_MaxPooling_1 (MaxPooling2D (None, 14, 14, 32)      

In [31]:
def build_cgan(generator, discriminator, z_dim):
    # Random noise vector z
    discriminator.trainable = False
    z = Input(shape=(z_dim,))

    # Image label
    label = Input(shape=(1,))

    # Generated image for that label
    img = generator([z, label])

    classification = discriminator([img, label])

    # Combined Generator -> Discriminator model
    # G([z, lablel]) = x*
    # D(x*) = classification
    model = Model([z, label], classification)
    
    plot_model(model, './cgan.jpg', show_shapes=True, show_dtype=True)

    return model

In [32]:
cgan = build_cgan(generator, discriminator, z_dim)
cgan.compile(loss='binary_crossentropy', optimizer='adam')

In [33]:
cgan.summary()

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_17 (InputLayer)           [(None, 100)]        0                                            
__________________________________________________________________________________________________
input_18 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
model_4 (Functional)            (None, 28, 28, 1)    7164997     input_17[0][0]                   
                                                                 input_18[0][0]                   
__________________________________________________________________________________________________
model_5 (Functional)            (None, 1)            1186961     model_4[0][0]              

In [34]:
def sample_images(epoch, image_grid_rows=2, image_grid_columns=2):
    # Sample random noise
    z = np.random.normal(0, 1, (image_grid_rows * image_grid_columns, z_dim))

    # Get image labels 0-9
    labels = np.arange(0, 4).reshape(-1, 1)

    # Generate images from random noise
    gen_imgs = generator.predict([z, labels])

    # Set image grid
    fig, axs = plt.subplots(image_grid_rows, image_grid_columns, )

    cnt = 0
    for i in range(image_grid_rows):
        for j in range(image_grid_columns):
            # Rescale image pixel values to [0, 1]
            img = (gen_imgs[cnt] + 1) / 2.0
            axs[i, j].imshow(img)
            axs[i, j].axis('off')
            axs[i, j].set_title("Digit: %d" % labels[cnt])
            cnt += 1
    filename = f'./generated_plot_e%03d.jpg' % (epoch + 1)
    plt.savefig(filename)
    plt.close()

In [35]:
accuracies = []
losses = []

def train(epochs, batch_size, sample_interval, img, labels):
    (X_train, y_train) = img, labels

    bat_per_epo = int(X_train.shape[0] / batch_size)
    half_batch = int(batch_size / 2)
    real = np.ones((batch_size, 1)) # np.asarray([.9] * batch_size)
    fake = np.zeros((batch_size, 1)) # np.asarray([.1] * batch_size)

    for epoch in range(epochs):
        g_loss = 0
        d_loss = []
        for j in range(bat_per_epo):
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs, labels = X_train[idx], y_train[idx]

            # Generate a batch of fake images
            z = np.random.normal(0, 1, (batch_size, z_dim))
            gen_imgs = generator.predict([z, labels])

            # Train the Discriminator
            d_loss_real = discriminator.train_on_batch([imgs, labels], real)
            d_loss_fake = discriminator.train_on_batch([gen_imgs, labels], fake)
#             d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            # Generate a batch of noise vectors
            z = np.random.normal(0, 1, (batch_size, z_dim))

            # Get a batch of random labels
            labels = np.random.randint(0, num_class, batch_size).reshape(-1, 1)

            # Train the Generator
            g_loss = cgan.train_on_batch([z, labels], real)
            print('>%d, %d/%d, [D loss_f: %f, acc_f: %.2f%%, loss_r:%f, acc_r:%.2f%%] [G loss: %f]' % (epoch + 1, j + 1, bat_per_epo, d_loss_fake[0], 100 * d_loss_fake[1],d_loss_real[0], 100 * d_loss_real[1], g_loss))

        if (epoch + 1) % sample_interval == 0:
            # Output training progress

            # Save losses and accuracies so they can be plotted after training
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            losses.append((d_loss[0], g_loss))
            accuracies.append(100 * d_loss[1])

            # Output sample of generated images
            sample_images(epoch)

In [38]:
train(epochs, batch_size, sample_interval, img, labels)

2021-10-11 14:48:30.027851: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-10-11 14:48:32.050816: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8204


>1, 1/2187, [D loss_f: 16.490730, acc_f: 0.00%, loss_r:0.924828, acc_r:34.38%] [G loss: 0.662958]
>1, 2/2187, [D loss_f: 0.789876, acc_f: 65.62%, loss_r:0.000105, acc_r:100.00%] [G loss: 0.671825]
>1, 3/2187, [D loss_f: 0.002703, acc_f: 100.00%, loss_r:0.817080, acc_r:56.25%] [G loss: 0.665148]
>1, 4/2187, [D loss_f: 0.013173, acc_f: 100.00%, loss_r:0.580538, acc_r:65.62%] [G loss: 0.631284]
>1, 5/2187, [D loss_f: 0.056050, acc_f: 100.00%, loss_r:0.011472, acc_r:100.00%] [G loss: 0.591497]
>1, 6/2187, [D loss_f: 0.240253, acc_f: 90.62%, loss_r:0.002361, acc_r:100.00%] [G loss: 0.567969]
>1, 7/2187, [D loss_f: 0.027347, acc_f: 100.00%, loss_r:0.001309, acc_r:100.00%] [G loss: 0.551710]
>1, 8/2187, [D loss_f: 0.002678, acc_f: 100.00%, loss_r:0.007204, acc_r:100.00%] [G loss: 0.547484]
>1, 9/2187, [D loss_f: 0.001079, acc_f: 100.00%, loss_r:0.004907, acc_r:100.00%] [G loss: 0.548252]
>1, 10/2187, [D loss_f: 0.000519, acc_f: 100.00%, loss_r:0.009247, acc_r:100.00%] [G loss: 0.540848]
>1, 1

>1, 83/2187, [D loss_f: 0.000028, acc_f: 100.00%, loss_r:0.000039, acc_r:100.00%] [G loss: 0.000087]
>1, 84/2187, [D loss_f: 0.000027, acc_f: 100.00%, loss_r:0.000005, acc_r:100.00%] [G loss: 0.000078]
>1, 85/2187, [D loss_f: 0.000030, acc_f: 100.00%, loss_r:0.000009, acc_r:100.00%] [G loss: 0.000062]
>1, 86/2187, [D loss_f: 0.000031, acc_f: 100.00%, loss_r:0.000006, acc_r:100.00%] [G loss: 0.000045]
>1, 87/2187, [D loss_f: 0.000026, acc_f: 100.00%, loss_r:0.000416, acc_r:100.00%] [G loss: 0.000037]
>1, 88/2187, [D loss_f: 0.000029, acc_f: 100.00%, loss_r:0.000055, acc_r:100.00%] [G loss: 0.000029]
>1, 89/2187, [D loss_f: 0.000029, acc_f: 100.00%, loss_r:0.000050, acc_r:100.00%] [G loss: 0.000022]
>1, 90/2187, [D loss_f: 0.000017, acc_f: 100.00%, loss_r:0.000043, acc_r:100.00%] [G loss: 0.000019]
>1, 91/2187, [D loss_f: 0.000025, acc_f: 100.00%, loss_r:0.000034, acc_r:100.00%] [G loss: 0.000015]
>1, 92/2187, [D loss_f: 0.000020, acc_f: 100.00%, loss_r:0.000703, acc_r:100.00%] [G loss: 

>1, 165/2187, [D loss_f: 0.000012, acc_f: 100.00%, loss_r:0.000004, acc_r:100.00%] [G loss: 0.000000]
>1, 166/2187, [D loss_f: 0.000010, acc_f: 100.00%, loss_r:0.000004, acc_r:100.00%] [G loss: 0.000000]
>1, 167/2187, [D loss_f: 0.000010, acc_f: 100.00%, loss_r:0.000032, acc_r:100.00%] [G loss: 0.000000]
>1, 168/2187, [D loss_f: 0.000010, acc_f: 100.00%, loss_r:0.000006, acc_r:100.00%] [G loss: 0.000000]
>1, 169/2187, [D loss_f: 0.000014, acc_f: 100.00%, loss_r:0.000121, acc_r:100.00%] [G loss: 0.000000]
>1, 170/2187, [D loss_f: 0.000013, acc_f: 100.00%, loss_r:0.000018, acc_r:100.00%] [G loss: 0.000000]
>1, 171/2187, [D loss_f: 0.000012, acc_f: 100.00%, loss_r:0.000034, acc_r:100.00%] [G loss: 0.000000]
>1, 172/2187, [D loss_f: 0.000010, acc_f: 100.00%, loss_r:0.000009, acc_r:100.00%] [G loss: 0.000000]
>1, 173/2187, [D loss_f: 0.000011, acc_f: 100.00%, loss_r:0.000221, acc_r:100.00%] [G loss: 0.000000]
>1, 174/2187, [D loss_f: 0.000009, acc_f: 100.00%, loss_r:0.000009, acc_r:100.00%]

>1, 247/2187, [D loss_f: 0.000007, acc_f: 100.00%, loss_r:0.000012, acc_r:100.00%] [G loss: 0.000000]
>1, 248/2187, [D loss_f: 0.000007, acc_f: 100.00%, loss_r:0.000005, acc_r:100.00%] [G loss: 0.000000]
>1, 249/2187, [D loss_f: 0.000006, acc_f: 100.00%, loss_r:0.000009, acc_r:100.00%] [G loss: 0.000000]
>1, 250/2187, [D loss_f: 0.000007, acc_f: 100.00%, loss_r:0.000004, acc_r:100.00%] [G loss: 0.000000]
>1, 251/2187, [D loss_f: 0.000006, acc_f: 100.00%, loss_r:0.000003, acc_r:100.00%] [G loss: 0.000000]
>1, 252/2187, [D loss_f: 0.000009, acc_f: 100.00%, loss_r:0.000004, acc_r:100.00%] [G loss: 0.000000]
>1, 253/2187, [D loss_f: 0.000005, acc_f: 100.00%, loss_r:0.000019, acc_r:100.00%] [G loss: 0.000000]
>1, 254/2187, [D loss_f: 0.000005, acc_f: 100.00%, loss_r:0.000003, acc_r:100.00%] [G loss: 0.000000]
>1, 255/2187, [D loss_f: 0.000008, acc_f: 100.00%, loss_r:0.000045, acc_r:100.00%] [G loss: 0.000000]
>1, 256/2187, [D loss_f: 0.000004, acc_f: 100.00%, loss_r:0.000002, acc_r:100.00%]

...

>6, 564/2187, [D loss_f: 0.000000, acc_f: 100.00%, loss_r:0.000000, acc_r:100.00%] [G loss: 0.000000]
>6, 565/2187, [D loss_f: 0.000000, acc_f: 100.00%, loss_r:0.000000, acc_r:100.00%] [G loss: 0.000000]
>6, 566/2187, [D loss_f: 0.000000, acc_f: 100.00%, loss_r:0.000000, acc_r:100.00%] [G loss: 0.000000]
>6, 567/2187, [D loss_f: 0.000000, acc_f: 100.00%, loss_r:0.000000, acc_r:100.00%] [G loss: 0.000000]
>6, 568/2187, [D loss_f: 0.000000, acc_f: 100.00%, loss_r:0.000000, acc_r:100.00%] [G loss: 0.000000]
>6, 569/2187, [D loss_f: 0.000000, acc_f: 100.00%, loss_r:0.000000, acc_r:100.00%] [G loss: 0.000000]
>6, 570/2187, [D loss_f: 0.000000, acc_f: 100.00%, loss_r:0.000000, acc_r:100.00%] [G loss: 0.000000]
>6, 571/2187, [D loss_f: 0.000000, acc_f: 100.00%, loss_r:0.000000, acc_r:100.00%] [G loss: 0.000000]
>6, 572/2187, [D loss_f: 0.000000, acc_f: 100.00%, loss_r:0.000000, acc_r:100.00%] [G loss: 0.000000]
>6, 573/2187, [D loss_f: 0.000000, acc_f: 100.00%, loss_r:0.000000, acc_r:100.00%]

>6, 646/2187, [D loss_f: 0.000000, acc_f: 100.00%, loss_r:0.000000, acc_r:100.00%] [G loss: 0.000000]
>6, 647/2187, [D loss_f: 0.000000, acc_f: 100.00%, loss_r:0.000000, acc_r:100.00%] [G loss: 0.000000]
>6, 648/2187, [D loss_f: 0.000000, acc_f: 100.00%, loss_r:0.000000, acc_r:100.00%] [G loss: 0.000000]
>6, 649/2187, [D loss_f: 0.000000, acc_f: 100.00%, loss_r:0.000000, acc_r:100.00%] [G loss: 0.000000]
>6, 650/2187, [D loss_f: 0.000000, acc_f: 100.00%, loss_r:0.000000, acc_r:100.00%] [G loss: 0.000000]
>6, 651/2187, [D loss_f: 0.000000, acc_f: 100.00%, loss_r:0.000000, acc_r:100.00%] [G loss: 0.000000]
>6, 652/2187, [D loss_f: 0.000000, acc_f: 100.00%, loss_r:0.000000, acc_r:100.00%] [G loss: 0.000000]
>6, 653/2187, [D loss_f: 0.000000, acc_f: 100.00%, loss_r:0.000000, acc_r:100.00%] [G loss: 0.000000]
>6, 654/2187, [D loss_f: 0.000000, acc_f: 100.00%, loss_r:0.000000, acc_r:100.00%] [G loss: 0.000000]
>6, 655/2187, [D loss_f: 0.000000, acc_f: 100.00%, loss_r:0.000000, acc_r:100.00%]

>6, 728/2187, [D loss_f: 0.000000, acc_f: 100.00%, loss_r:0.000000, acc_r:100.00%] [G loss: 0.000000]
>6, 729/2187, [D loss_f: 0.000000, acc_f: 100.00%, loss_r:0.000000, acc_r:100.00%] [G loss: 0.000000]
>6, 730/2187, [D loss_f: 0.000000, acc_f: 100.00%, loss_r:0.000000, acc_r:100.00%] [G loss: 0.000000]
>6, 731/2187, [D loss_f: 0.000000, acc_f: 100.00%, loss_r:0.000000, acc_r:100.00%] [G loss: 0.000000]
>6, 732/2187, [D loss_f: 0.000000, acc_f: 100.00%, loss_r:0.000000, acc_r:100.00%] [G loss: 0.000000]
>6, 733/2187, [D loss_f: 0.000000, acc_f: 100.00%, loss_r:0.000000, acc_r:100.00%] [G loss: 0.000000]
>6, 734/2187, [D loss_f: 0.000000, acc_f: 100.00%, loss_r:0.000000, acc_r:100.00%] [G loss: 0.000000]
>6, 735/2187, [D loss_f: 0.000000, acc_f: 100.00%, loss_r:0.000000, acc_r:100.00%] [G loss: 0.000000]
>6, 736/2187, [D loss_f: 0.000000, acc_f: 100.00%, loss_r:0.000000, acc_r:100.00%] [G loss: 0.000000]
>6, 737/2187, [D loss_f: 0.000000, acc_f: 100.00%, loss_r:0.000000, acc_r:100.00%]

>6, 810/2187, [D loss_f: 0.000000, acc_f: 100.00%, loss_r:0.000000, acc_r:100.00%] [G loss: 0.000000]
>6, 811/2187, [D loss_f: 0.000000, acc_f: 100.00%, loss_r:0.000000, acc_r:100.00%] [G loss: 0.000000]
>6, 812/2187, [D loss_f: 0.000000, acc_f: 100.00%, loss_r:0.000000, acc_r:100.00%] [G loss: 0.000000]
>6, 813/2187, [D loss_f: 0.000000, acc_f: 100.00%, loss_r:0.000000, acc_r:100.00%] [G loss: 0.000000]
>6, 814/2187, [D loss_f: 0.000000, acc_f: 100.00%, loss_r:0.000000, acc_r:100.00%] [G loss: 0.000000]
>6, 815/2187, [D loss_f: 0.000000, acc_f: 100.00%, loss_r:0.000000, acc_r:100.00%] [G loss: 0.000000]


KeyboardInterrupt: 

In [19]:
pred = discriminator.predict([img[-3:], labels[-3:]])

In [20]:
pred

array([[0.47253737],
       [0.47338888],
       [0.5593785 ]], dtype=float32)

In [24]:
y_true = [[.9], [.9], [.9]]
y_pred2= [[0.989999], [0.989999], [0.9000]]
bce = tf.keras.losses.BinaryCrossentropy()
bce(y_true, y_pred2).numpy()

0.42139575

In [ ]:
m = keras.metrics.BinaryAccuracy()
m.update_state([[1], [1], [0], [0]], [[0.98], [1], [0], [0.6]])
m.result().numpy()

In [ ]:
m = keras.metrics.BinaryAccuracy()
m.update_state([[.9], [.9], [.9]], [[0.6650624 ],[0.67847896],[0.66430885]])
m.result().numpy()